<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/IsopropanolSynthesis_che3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che3.ipynb -O che3.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/lsq_root.ipynb -O lsq_root.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook(): 
    from che3 import DotDict, Range, RangeArray, d2nt, Props
    from lsq_root import lsq_root

In [3]:
import jax
import jax.numpy as jnp

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [4]:
# Static parameters (Total feed, feed mole fradtions, feed temperature and )
# 1. propene
# 2. diisopropyl ether
# 3. isopropanol
# 4. water
# Rx1:  C3H6 + H2O -> C3H8O
# Rx2:  2C3H8O -> C6H14 + H2O 

#Simple version - Overall mass balance only

e = DotDict()
e.alkeneF = 100.
e.waterF = 100.

e.alkeney = jnp.array([1., 0., 0., 0.])
e.waterx = jnp.array([0., 0., 0., 1.])

e.rx1 = jnp.array([-1, 0, 1, -1])
e.rx2 = jnp.array([0, 1, -2, 1])

e.rx1_extent = Range(e.alkeneF/2, 0., e.alkeneF)
e.rx2_extent = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.alcoholP = Range(e.alkeneF/2, 0., e.alkeneF)  
e.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

e.etherP = Range(e.alkeneF/2, 0., e.alkeneF)
e.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

In [5]:
x0, lb, ub, x2nt, wrap, x2unk, const  = d2nt(e)

In [6]:
def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx 
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r

In [7]:
eqs_wrapped = wrap(eqs)
eqs_wrapped(x0)

DeviceArray([ 49.975, -24.25 , -47.9  ,  72.175], dtype=float64)

In [8]:
sol=lsq_root(eqs_wrapped, x0)
print(sol)

 active_mask: array([0., 0., 0., 0.])
        cost: 3.7992236653254967e-19
         fun: array([-3.94241272e-10,  1.29940503e-12,  1.30654598e-10,  7.66385178e-10])
        grad: array([-2.41489308e-10,  5.06375386e-10, -1.64973004e-10, -4.57765683e-12])
         jac: <4x4 _CustomLinearOperator with dtype=float64>
     message: '`gtol` termination condition is satisfied.'
        nfev: 4
        njev: 4
  optimality: 5.06375386066793e-10
      status: 1
     success: True
           x: array([99.9975704 ,  5.23773658, 95.14322505,  4.85920455])


In [9]:
nt=x2nt(sol.x)
nt._asdict()

OrderedDict([('alkeneF', DeviceArray(100., dtype=float64)),
             ('waterF', DeviceArray(100., dtype=float64)),
             ('alkeney', DeviceArray([1., 0., 0., 0.], dtype=float64)),
             ('waterx', DeviceArray([0., 0., 0., 1.], dtype=float64)),
             ('rx1', DeviceArray([-1,  0,  1, -1], dtype=int64)),
             ('rx2', DeviceArray([ 0,  1, -2,  1], dtype=int64)),
             ('rx1_extent', DeviceArray(99.9975704, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23773658, dtype=float64)),
             ('alcoholP', DeviceArray(95.14322505, dtype=float64)),
             ('alcoholPx',
              DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64)),
             ('etherP', DeviceArray(4.85920455, dtype=float64)),
             ('etherPx',
              DeviceArray([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64))])

In [10]:
unk=x2unk(sol.x)
unk._asdict()

OrderedDict([('rx1_extent', DeviceArray(99.9975704, dtype=float64)),
             ('rx2_extent', DeviceArray(5.23773658, dtype=float64)),
             ('alcoholP', DeviceArray(95.14322505, dtype=float64)),
             ('etherP', DeviceArray(4.85920455, dtype=float64))])

In [11]:
const

_(alkeneF=DeviceArray(100., dtype=float64), waterF=DeviceArray(100., dtype=float64), alkeney=DeviceArray([1., 0., 0., 0.], dtype=float64), waterx=DeviceArray([0., 0., 0., 1.], dtype=float64), rx1=DeviceArray([-1,  0,  1, -1], dtype=int64), rx2=DeviceArray([ 0,  1, -2,  1], dtype=int64), alcoholPx=DeviceArray([0.   , 0.005, 0.94 , 0.055], dtype=float64), etherPx=DeviceArray([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64))

In [12]:
_, r= eqs(x2nt(sol.x))
r

{'alkene': DeviceArray([100.,   0.,   0.,   0.], dtype=float64),
 'water': DeviceArray([  0.,   0.,   0., 100.], dtype=float64),
 'ether': DeviceArray([2.42960227e-03, 4.76202046e+00, 8.74656819e-02,
              7.28880682e-03], dtype=float64),
 'alcohol': DeviceArray([ 0.        ,  0.47571613, 89.43463155,  5.23287738], dtype=float64)}

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [19]:
# Include internal flows (recycle)


d=DotDict()
d.alkeneF = 100.
d.waterF = 100.

d.alkeney = jnp.array([1., 0., 0., 0.])
d.waterx = jnp.array([0., 0., 0., 1.])

d.rx1 = jnp.array([-1, 0, 1, -1])
d.rx2 = jnp.array([0, 1, -2, 1])

d.rx1_extent = Range(d.alkeneF/2, 0., d.alkeneF)
d.rx2_extent = Range(d.alkeneF/4, 0., d.alkeneF/2)

d.alcoholP = Range(d.alkeneF/2, 0., d.alkeneF)  
d.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

d.etherP = Range(d.alkeneF/2, 0., d.alkeneF)
d.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

d.WARFR = 12. #water_alkene_reactor_feed_ratio

d.D1F = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.D1Fx = RangeArray([0.001, 0.03, 0.07, 0.9],0,1)

d.D1K = jnp.array([100., 10., 5., 0.2])
d.D1Vy = RangeArray([0.01, 0.299, 0.3, 0.3],0,1)
d.D1Lx = RangeArray([0.0001, 0.1, 0.1, 0.8],0,1)
d.D1V = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)
d.D1L = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)

d.decantW = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.decantWx = jnp.array([0.0001, 0.0005, 0.0014, 0.98]) 


In [20]:
x0, lb, ub, x2nt, wrap, x2unk, const  = d2nt(d)

In [22]:
def eqs2(d):
    r=DotDict()
    r.MOLFRAC = jnp.ravel(jnp.sum(jnp.stack([d.D1Fx, d.D1Vy, d.D1Lx]),axis=1)-1.)
    r.reactor_feed = d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1L*d.D1Lx + d.decantW*d.decantWx

    r.REACTOR = r.reactor_feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - d.D1F*d.D1Fx

    r.FEED_RATIO = jnp.atleast_1d((d.waterF+d.D1L+d.decantW)/d.alkeneF - d.WARFR)
    
    r.D1 = d.D1F*d.D1Fx - d.D1V*d.D1Vy - d.D1L*d.D1Lx
    r.D1K = d.D1Vy/d.D1Lx - d.D1K

    r.AZ = d.D1V*d.D1Vy - d.etherP*d.etherPx - d.alcoholP*d.alcoholPx - d.decantW*d.decantWx

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return jnp.concatenate([r.MOLFRAC, r.REACTOR, r.FEED_RATIO, r.D1, r.D1K, r.AZ]), r


In [23]:
eqs2_wrapped = wrap(eqs2)

In [ ]:
sol=lsq_root(eqs2_wrapped, x0, bounds=[lb,ub],dense=True, max_nfev=1000,ftol=1e-14, gtol=1e-14, xtol=1e-14)
sol

In [ ]:
unk=x2unk(sol.x)
unk._asdict()

In [ ]:
eqs2_wrapped(sol.x)